In [1]:
import itertools
import os
import numpy as np
import pandas as pd
import pickle

Since these surveys were designed, we've made a couple changes to the PKL calculations (see changes.md). The old calculations are saved to make the surveys reproducible.

In [3]:
# Load data
# Created using pyspan/scripts/get_interchangeable_terms.py
with open("version_used_for_LoP_2AFC_2_cos_sim_partial_kls-unigrams", "rb") as rfh:
    df = pickle.load(rfh)

In [4]:
words = zip(df["dterm"], df["rterm"])
words.remove(("bush","clinton"))
words.remove(("bush","obama"))
words.remove(("congressman","steve"))
words.remove(("trump","obama"))
words.remove(("congressman","mike"))
words.remove(("congressman","tom"))
words.remove(("congressman","chris"))
words.remove(("congressman","randy"))
words.remove(("congressman","phil"))
words.remove(("congressman","joe"))
words.remove(("bush","reagan"))
words.remove(("zinser","comey"))
words.remove(("zinser","mueller"))
words.remove(("congressman","bob"))
words.remove(("representative","steve"))
words.remove(("congressman","jay"))
words.remove(("trump","clinton"))
words.remove(("martin","dr"))
words.remove(("congressman","ralph"))
words.remove(("representative","mike"))
words.remove(("cr","ndaa"))
words.remove(("donald","steve"))
words.remove(("representative","chris"))
words.remove(("congressman","kevin"))
words.remove(("representative","tom"))
words.remove(("center","inc"))
words.remove(("stokes", "ralph"))

In [5]:
dups = []
for i in range(1000):
    if any([ w in set(itertools.chain(*words[:i])) for w in words[i] ]):
        dups.append(words[i])
    if ( "ret" in words[i] or "jr" in words[i] or "boko" in words[i] or "haram" in words[i] or
         "rep" in words[i] ):
        dups.append(words[i])
dups = set(dups)
        
for dup in dups:
    words.remove(dup)

In [6]:
words = words[:88]

In [8]:
# Include opposite-valenced words
df_antonyms = pd.read_csv("version_used_for_LoP_2AFC_2_antonyms_modified_with-pkls.csv")
antonyms = zip(df_antonyms["0"], df_antonyms["1"])[:10]
words += antonyms

In [9]:
pickle.dump(pd.DataFrame(words), open("survey_terms.pkl", "wb"))

In [10]:
wfh1 = open("survey2_1.txt", "w")
wfh2 = open("survey2_2.txt", "w")
atc1 = open("attention_check2_1.txt", "w")
atc2 = open("attention_check2_2.txt", "w")
wfh1.write("[[Block:Word_pairs_1]]")
wfh2.write("[[Block:Word_pairs_2]]")
# Instructions to be included at the top of every page
instructions = """
    
LOOPN. For each word pair, please guess which is indicative that the speaker is a ${e://Field/Condition}. Your responses will be most helpful if you go with your first impression, so we ask that you spend no more than 5 seconds on each word pair---but feel free to take breaks in between pages!

"""

# All words will be presented once, and two will be randomly selected as attention checks
for i, words_ in enumerate(words):
    order = np.random.choice((0, 1))
    word1, word2 = words_ if order==0 else(words_[1], words_[0])
    if i%20 == 0:
        wfh1.write(instructions.replace("LOOPN", str(1000+i)))
    wfh1.write("""

{}.\t

{}
{}

""".format(i+1, word1, word2))
    atc1.write("""

{}.\t 

{}
{}

""".format(i+99, word2, word1))
    if i%20 == 0:
        wfh2.write(instructions.replace("LOOPN", str(1000+i)))
    wfh2.write("""

{}.\t 

{}
{}

""".format(i+1, word2, word1))
    atc2.write("""

{}.\t

{}
{}

""".format(i+99, word1, word2))
    
wfh1.write("""

98.1. BREAK

""")
wfh2.write("""

98.1. BREAK

""")

In [11]:
wfh1.close()
wfh2.close()
atc1.close()
atc2.close()

In [12]:
# Merge attention checks with main blocks
os.system("cat attention_check2_1.txt >> survey2_1.txt")
os.system("cat attention_check2_2.txt >> survey2_2.txt")

0